<a href="https://colab.research.google.com/github/mantasbandonis/denoiser/blob/main/denoiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mantasbandonis/denoiser

Cloning into 'denoiser'...
remote: Enumerating objects: 13432, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 13432 (delta 166), reused 389 (delta 148), pack-reused 13009
Receiving objects: 100% (13432/13432), 287.58 MiB | 35.88 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [16]:
import os
os.getcwd()

'/content/denoiser'

In [2]:
%cd /content/denoiser

/content/denoiser


In [16]:
#%cd /content/denoiser/dataset/debug

/content/denoiser/dataset/debug


In [4]:
!pip install -r requirements.txt

  Cloning https://github.com/ludlows/python-pesq to /tmp/pip-install-ihqjcs9o/pesq_493d3c81ea774c08aa46cdcadbfa47e5
  Running command git clone -q https://github.com/ludlows/python-pesq /tmp/pip-install-ihqjcs9o/pesq_493d3c81ea774c08aa46cdcadbfa47e5
     |████████████████████████████████| 72 kB 613 kB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 14.6 MB 26.2 MB/s 
     |████████████████████████████████| 3.2 MB 44.2 MB/s 
     |████████████████████████████████| 753.2 MB 15 kB/s 
     |████████████████████████████████| 58 kB 7.0 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 41 kB 686 kB/s 
     |████████████████████████████████| 41 kB 175 kB/s 
     |████████████████████████████████| 40 kB 6.9 MB/s 
     |████████████████████████████████| 40 kB 6.2 MB/s 
  Created wheel for pesq: filename=pesq-0.0.3-cp37-cp37m-linux_x86_64.whl size=211399 sha256=f9b1470131d06615ab99f1b252df418d08

In [18]:
!pip install denoiser

     |████████████████████████████████| 48 kB 4.1 MB/s 
  Created wheel for denoiser: filename=denoiser-0.1.4-py3-none-any.whl size=42782 sha256=c6865ce1097fe04c740962b4dfd2863dd04c7945680dcf2f9537c4cd306319f3
  Stored in directory: /root/.cache/pip/wheels/76/d9/3d/f34ed4201f8090d6c2adf117ba05acec0c4635d0ed7416e0d9
Successfully built denoiser


In [5]:
!python -m denoiser.audio

[]

In [4]:
%mkdir egs/debug/tr
!python -m denoiser.audio dataset/debug/noisy > egs/debug/tr/noisy.json
!python -m denoiser.audio dataset/debug/clean > egs/debug/tr/clean.json

In [6]:
!pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 5.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [8]:
! ./train.py

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Mus

In [8]:
import wave
import audioop

In [26]:
def downsampleWav(src, dst, inrate=11025, outrate=16000, inchannels=2, outchannels=1):
    if not os.path.exists(src):
        print('Source not found!')
        return False

    if not os.path.exists(os.path.dirname(dst)):
        os.makedirs(os.path.dirname(dst))

    try:
        s_read = wave.open(src, 'rb')
        s_write = wave.open(dst, 'wb')
    except:
        print('Failed to open files!')
        return False

    n_frames = s_read.getnframes()
    data = s_read.readframes(n_frames)

    try:
        converted = audioop.ratecv(data, 2, inchannels, inrate, outrate, None)
        if outchannels == 1:
            converted = audioop.tomono(converted[0], 2, 1, 0)
    except:
        print('Failed to downsample wav')
        return False

    try:
        s_write.setparams((outchannels, 2, outrate, 0, 'NONE', 'Uncompressed'))
        s_write.writeframes(converted)
    except:
        print('Failed to write wav')
        return False

    try:
        s_read.close()
        s_write.close()
    except:
        print('Failed to close wav files')
        return False

    return True

In [27]:
os.path.exists("/content/denoiser/dataset/debug/clean")

True

In [28]:
src = "/content/denoiser/dataset/debug/clean"
dst = "/content/denoiser/dataset/debug/clean"

In [36]:
inrate=11025
outrate=8000
inchannels=2
outchannels=1

In [35]:
import librosa

In [ ]:
for filename in glob.glob(os.path.join(path, '*.wav')):  
  samples, sample_rate = librosa.load(filename, sr = 16000)
  

In [37]:
import wave, os, glob
path = '/content/denoiser/dataset/debug/clean'



for filename in glob.glob(os.path.join(path, '*.wav')):  
    try:
        s_read = wave.open(filename, 'rb')
        s_write = wave.open(filename, 'wb')
    except:
        print('Failed to open files!')

    n_frames = s_read.getnframes()
    data = s_read.readframes(n_frames)

    try:
        converted = audioop.ratecv(data, 2, inchannels, inrate, outrate, None)
        if outchannels == 1:
            converted = audioop.tomono(converted[0], 2, 1, 0)
    except:
        print('Failed to downsample wav')

    try:
        s_write.setparams((outchannels, 2, outrate, 0, 'NONE', 'Uncompressed'))
        s_write.writeframes(converted)
    except:
        print('Failed to write wav')

    try:
        s_read.close()
        s_write.close()
    except:
        print('Failed to close wav files')

Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav
Failed to downsample wav


In [30]:
for fil in src:
  downsampleWav(fil, fil, inrate=11025, outrate=16000, inchannels=2, outchannels=1)

Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!
Failed to open files!
Source not found!
Source not found!
Source not found!
Source not found!
Source not found!


In [5]:
!./train.py continue_pretrained=dns48

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Must select either 8000 or 16000.
       Note there is narrow band (nb) mode only when sampling rate is 8000Hz.
       

       Run model on reference ref and degraded deg
       Sample rate (fs) - No default. Mus

In [17]:
!git config --global user.email "mantas.bandonis@outlook.com"

In [18]:
!git config --global user.name "Mantas Bandonis"

In [19]:
!git add . && git commit -m "commit changes"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
